In [1]:
import logging
import sys
import pandas as pd
import numpy as np
from numba import jit
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [2]:
import util

Loading BokehJS ...

In [3]:
util.seed(16)

## Data preprocessing

In [4]:
df = pd.read_csv('../Dataset/output/SP500.csv')
df['Date'] = pd.to_datetime(df['Date'], yearfirst=True)



In [5]:
train_dict={}
for name, sdf in df.groupby('Name'):
    sdf.set_index('Date').sort_index()
    train_df, test_df= train_test_split(sdf, test_size=0.2, shuffle=False)
    train_dict[name]=train_df
      


In [6]:
train_dict.get('AAPL')

,Date,open,high,low,close,adjclose,volume,Name
51423,2013-02-08,16.928572,17.100357,16.723213,16.963572,14.591927,633158400.0,AAPL
51424,2013-02-11,17.017857,17.319286,16.901787,17.140356,14.743990,517490400.0,AAPL
51425,2013-02-12,17.125357,17.227858,16.705000,16.710714,14.374419,609053200.0,AAPL
51426,2013-02-13,16.686071,16.915714,16.543571,16.678928,14.347075,475207600.0,AAPL
51427,2013-02-14,16.590000,16.844286,16.572144,16.663929,14.334173,355275200.0,AAPL
...,...,...,...,...,...,...,...,...
52425,2017-02-01,31.757500,32.622501,31.752501,32.187500,29.955246,447940000.0,AAPL
52426,2017-02-02,31.995001,32.347500,31.945000,32.132500,29.904062,134841600.0,AAPL
52427,2017-02-03,32.077499,32.297501,32.040001,32.270000,30.032024,98029200.0,AAPL
52428,2017-02-06,32.282501,32.625000,32.224998,32.572498,30.313543,107383600.0,AAPL


In [7]:
min_max_scaler = MinMaxScaler()
for d in train_dict.values():
    d[['scaled_adjclose']]=min_max_scaler.fit_transform(d[['adjclose']])

In [8]:
train_dict.get('AAPL')

,Date,open,high,low,close,adjclose,volume,Name,scaled_adjclose
51423,2013-02-08,16.928572,17.100357,16.723213,16.963572,14.591927,633158400.0,AAPL,0.139450
51424,2013-02-11,17.017857,17.319286,16.901787,17.140356,14.743990,517490400.0,AAPL,0.147623
51425,2013-02-12,17.125357,17.227858,16.705000,16.710714,14.374419,609053200.0,AAPL,0.127759
51426,2013-02-13,16.686071,16.915714,16.543571,16.678928,14.347075,475207600.0,AAPL,0.126289
51427,2013-02-14,16.590000,16.844286,16.572144,16.663929,14.334173,355275200.0,AAPL,0.125596
...,...,...,...,...,...,...,...,...,...
52425,2017-02-01,31.757500,32.622501,31.752501,32.187500,29.955246,447940000.0,AAPL,0.965234
52426,2017-02-02,31.995001,32.347500,31.945000,32.132500,29.904062,134841600.0,AAPL,0.962483
52427,2017-02-03,32.077499,32.297501,32.040001,32.270000,30.032024,98029200.0,AAPL,0.969361
52428,2017-02-06,32.282501,32.625000,32.224998,32.572498,30.313543,107383600.0,AAPL,0.984493


## Trend calculation using linear regression

In [9]:
uptrend ={}
downtrend ={}
sideway={}

In [10]:
def trend(f):
    f.index = f.index.map(pd.Timestamp.toordinal)
    data=f.reset_index()

    # create the model
    model = LinearRegression()

    # extract x and y from dataframe data
    x = data[['Date']]
    y = data[['scaled_adjclose']]

    # fit the mode
    model.fit(x, y)
    slope_ = model.coef_
    return slope_


In [11]:
for key,values in train_dict.items():
    slope= trend(values.set_index('Date').sort_index())
    if slope>0.00015 :
        uptrend[key]=values
    elif slope <-0.00015 :
        downtrend[key]=values
    elif slope<0.00015 and slope>-0.00015:
        sideway[key]=values

In [12]:
len(uptrend)

400

In [13]:
len(downtrend)

39

In [14]:
len(sideway)

51

## Sorting stocks based on liquidity

In [15]:
def sort_by_liquidity(trend_df):
    temp_dict={}
    for k,i in trend_df.items():
        temp_dict[k]=i['volume'].mean()
    s=sorted(temp_dict.items(), key=lambda x:x[1], reverse=True)
    df=pd.DataFrame(s,columns=['Name','mean_vol'])
    df.index.names = ['RowNo']
    return df

In [16]:
uptrend_df=sort_by_liquidity(uptrend)
downtrend_df=sort_by_liquidity(downtrend)
sideway_df=sort_by_liquidity(sideway)

## Save results to files

In [17]:
uptrend_df.to_csv('output/uptrend.csv')
downtrend_df.to_csv('output/downtrend.csv')
sideway_df.to_csv('output/sideway.csv')

## Visualize results

This section is just for visualizing a stock trend using linear regression. You can change the stock name for the function "read_candles_one" to visualize other stocks.

In [18]:
util.read_candles()
adf = util.read_candles_one('CVX')
train_df, test_df = train_test_split(adf, test_size=0.2, shuffle=False)
util.plot_candles(train_df)

In [19]:
train_df

,open,high,low,close,adjclose,volume
Date,,,,,,
2013-02-08,114.930000,115.779999,114.900002,115.639999,73.580597,4457200.0
2013-02-11,115.680000,115.839996,115.010002,115.639999,73.580597,3766300.0
2013-02-12,115.720001,116.839996,115.510002,116.500000,74.127808,4530600.0
2013-02-13,115.430000,115.839996,114.910004,115.529999,74.082916,4111000.0
2013-02-14,115.160004,116.260002,115.070000,115.709999,74.198318,5226400.0
...,...,...,...,...,...,...
2017-02-01,111.199997,111.779999,110.839996,111.000000,82.496742,5889300.0
2017-02-02,111.169998,112.239998,110.709999,112.209999,83.396034,6289000.0
2017-02-03,112.410004,114.279999,112.139999,113.570000,84.406815,6623500.0


In [20]:
min_max_scaler = MinMaxScaler()
train_df[['scaled_adjclose']]=min_max_scaler.fit_transform(train_df[['adjclose']])
train_df

,open,high,low,close,adjclose,volume,scaled_adjclose
Date,,,,,,,
2013-02-08,114.930000,115.779999,114.900002,115.639999,73.580597,4457200.0,0.594771
2013-02-11,115.680000,115.839996,115.010002,115.639999,73.580597,3766300.0,0.594771
2013-02-12,115.720001,116.839996,115.510002,116.500000,74.127808,4530600.0,0.608131
2013-02-13,115.430000,115.839996,114.910004,115.529999,74.082916,4111000.0,0.607035
2013-02-14,115.160004,116.260002,115.070000,115.709999,74.198318,5226400.0,0.609853
...,...,...,...,...,...,...,...
2017-02-01,111.199997,111.779999,110.839996,111.000000,82.496742,5889300.0,0.812464
2017-02-02,111.169998,112.239998,110.709999,112.209999,83.396034,6289000.0,0.834421
2017-02-03,112.410004,114.279999,112.139999,113.570000,84.406815,6623500.0,0.859100


In [21]:
train_df['x'] = train_df.index.map(pd.Timestamp.toordinal)
model = LinearRegression()
model.fit(train_df[['x']], train_df[['scaled_adjclose']])
print('intercept:', model.intercept_)
print('slope:', model.coef_)
trend = model.predict(train_df[['x']])
train_df['trend'] = min_max_scaler.inverse_transform(trend)

intercept: [107.12993018]
slope: [[-0.00014478]]


In [22]:
util.plot_trend(train_df)